In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [2]:
%load_ext line_profiler
%load_ext memory_profiler

In [3]:
import torch
from torch_geometric.data import Data, DataLoader
from torch.nn.functional import softmax, relu
from torch_geometric.nn import dense_diff_pool
from torch_geometric.transforms import BaseTransform
from torch import Tensor
from typing import Tuple


In [4]:
from utils.torchUtils.cpp_geometric import layers

In [5]:
import utils.torchUtils as gnn 
template = gnn.Dataset('../data/template',make_template=True, transform=gnn.HyperEdgeY())
dataset = gnn.Dataset('../data/MX_1200_MY_500-training',transform=template.transform)
# dataset = gnn.concat_dataset([f'eightb/pairing_methods/graph_net/data/{mass}-training' for mass in eightb.mass_list],transform=template.transform)

In [6]:
data = dataset[0]

In [7]:
data.edge_attr

tensor([[-5.5234e-01,  0.0000e+00, -1.7982e+00,  0.0000e+00,  0.0000e+00],
        [ 4.0799e+00,  1.7527e+00,  5.2261e-01,  1.2619e+00, -1.0581e+00],
        [ 3.2171e+00,  2.1086e+00,  6.4675e-01,  9.5174e-01, -1.2859e+00],
        [ 1.6645e+00,  2.3796e+00, -1.3210e-01, -3.0337e-01,  9.6153e-01],
        [ 1.8492e+00,  2.6050e+00, -7.6225e-02,  1.1100e+00,  6.6844e-01],
        [ 1.8195e+00,  2.7454e+00,  1.1676e+00,  1.1768e-01,  1.7499e+00],
        [ 2.3660e+00,  2.7479e+00,  7.5739e-01,  1.1356e+00,  1.2897e+00],
        [ 3.2519e+00,  2.7907e+00,  1.3241e+00,  1.6444e+00,  1.4531e+00],
        [ 5.6391e-01,  3.0807e+00, -2.3974e-01, -2.5428e-01, -9.0360e-01],
        [ 2.3933e+00,  3.2041e+00,  8.4535e-01,  2.1918e+00,  3.8324e-01],
        [ 4.0799e+00, -1.7527e+00,  5.2261e-01, -1.2619e+00,  1.0581e+00],
        [-6.7846e-01,  0.0000e+00, -1.7982e+00,  0.0000e+00,  0.0000e+00],
        [-2.6492e-01,  3.5591e-01, -1.2689e+00, -3.1015e-01, -2.2775e-01],
        [ 2.6223e+00,  6.

In [7]:
import utils.torchUtils as gnn

In [27]:
model = gnn.modelMap['golden_hyper_edge_conv'](dataset=template)

In [28]:
model(data)

(tensor([[-0.6873, -0.6990],
         [-0.6851, -0.7013],
         [-0.6849, -0.7014],
         [-0.6851, -0.7013],
         [-0.6889, -0.6974],
         [-0.6822, -0.7042],
         [-0.6829, -0.7035],
         [-0.6792, -0.7073],
         [-0.6836, -0.7028],
         [-0.6839, -0.7024]], grad_fn=<LogSoftmaxBackward>),
 tensor([[-0.6416, -0.7475],
         [-0.6387, -0.7507],
         [-0.6373, -0.7523],
         [-0.6430, -0.7460],
         [-0.6410, -0.7482],
         [-0.6385, -0.7510],
         [-0.6413, -0.7478],
         [-0.6403, -0.7490],
         [-0.6394, -0.7499],
         [-0.6385, -0.7510],
         [-0.6393, -0.7501],
         [-0.6371, -0.7525],
         [-0.6353, -0.7545],
         [-0.6382, -0.7513],
         [-0.6376, -0.7520],
         [-0.6368, -0.7529],
         [-0.6375, -0.7521],
         [-0.6379, -0.7516],
         [-0.6377, -0.7519],
         [-0.6379, -0.7516],
         [-0.6379, -0.7516],
         [-0.6356, -0.7542],
         [-0.6346, -0.7553],
         [-

In [20]:
x, edge_index, edge_attr, hyper_edge_index, hyper_edge_attr = data.x, \
    data.edge_index, data.edge_attr, \
    data.hyper_edge_index, data.hyper_edge_attr

In [21]:
x, edge_attr = model.edge_conv_1(x, edge_index, edge_attr)

In [22]:
x, edge_attr = model.g_relu_1(x, edge_attr)

In [24]:
x, hyper_edge_attr = model.hype_conv_1(x, hyper_edge_index, hyper_edge_attr)

In [25]:
x, edge_attr = model.edge_conv_2(x, edge_index, edge_attr)

In [26]:
x, edge_attr

(tensor([[ 28.9935,  13.0057, -14.7285,  ..., -14.4326,   7.9761,   2.1587],
         [ 14.2536,  -8.4263, -29.6611,  ...,   5.3014,   1.4122,  -4.7565],
         [  7.5626,  -7.1123, -23.8453,  ...,   4.0204,  -1.4770,  -6.8002],
         ...,
         [  2.8514, -22.5336, -31.9011,  ...,  11.6733,   8.1166,  -3.0087],
         [  7.2015,   0.0974, -22.8585,  ...,  -3.7897,   3.6067, -15.2909],
         [  7.8197,  -3.0585, -28.6274,  ...,  24.5565,   6.4888,  -0.5445]],
        grad_fn=<ScatterAddBackward>),
 tensor([[ 63.3110,  28.6871, -30.6734,  ..., -31.4522,  17.2765,   5.6220],
         [ 46.2618,   4.8230, -46.8142,  ...,  -9.7606,   9.4837,  -2.1926],
         [ 38.7554,   5.9557, -40.5492,  ..., -10.8073,   6.4356,  -4.1177],
         ...,
         [ 11.2271, -28.1211, -64.6501,  ...,  39.7408,  15.1025,  -2.3532],
         [ 15.0411,  -2.2754, -53.2683,  ...,  22.0169,  10.9625, -17.5840],
         [ 17.1559,  -6.7406, -61.5121,  ...,  53.9737,  13.5391,   0.2767]],
       